# README Examples

## Local config

In [1]:
# !pip install featuremesh==0.1.3

In [2]:
from libs.helpers.utils import get_featuremesh_config
from featuremesh import set_default, get_default, get_all_defaults

# API endpoints
fm_config = get_featuremesh_config()
set_default("registry.host", fm_config['registry.host'])
set_default("access.host", fm_config['access.host'])

debug_mode = get_default("debug_mode")
get_all_defaults()

{'registry.host': 'http://host.docker.internal:8001',
 'registry.path': '/v1/featureql',
 'registry.timeout': 30,
 'registry.verify_ssl': True,
 'serving.host': 'http://host.docker.internal:10090',
 'serving.path': '/v1/featureql',
 'serving.timeout': 30,
 'serving.verify_ssl': True,
 'access.host': 'http://host.docker.internal:8001',
 'access.path': '/v1/access',
 'access.path_auth': '/v1/auth',
 'access.timeout': 30,
 'access.verify_ssl': True,
 'identity_token': None,
 'access_token': None,
 'service_account_token': None,
 'debug_mode': False,
 'client': None,
 'show_sql': False,
 'hide_dataframe': False,
 'show_slt': False,
 'hook': None}

## Access management

In [3]:
from featuremesh import generate_access_token, decode_token

__YOUR_IDENTITY_TOKEN__ = fm_config['identity_token']
__YOUR_ACCESS_TOKEN__ = generate_access_token(identity_token=__YOUR_IDENTITY_TOKEN__, project='default')

decode_token(__YOUR_ACCESS_TOKEN__)

{'success': True,
 'token_info': {'token_type': 'USER_PROJECT_TOKEN',
  'user_id': '143eb6eb-727d-409e-9d31-31e8657abbed',
  'email': 'npissard@gmail.com',
  'issued_at': '2026-02-03T14:26:19+00:00',
  'expires_at': '2026-02-10T14:26:19+00:00',
  'is_expired': False,
  'is_invalidated': False,
  'project': {'id': 'e11a7aa4-1c91-4720-83c3-0043bb51e2dc',
   'name': '143eb6eb-727d-409e-9d31-31e8657abbed/default',
   'role': 'OWNER'}}}

## DuckDB setup

In [4]:
from featuremesh import OfflineClient, Backend
import duckdb

# Option 1: Using a persistent connection
_duckdb_conn = None

def get_duckdb_conn(storage_path: str = ":memory:"):
    """Get or create a DuckDB connection."""
    global _duckdb_conn
    if _duckdb_conn is None:
        _duckdb_conn = duckdb.connect(storage_path)
    return _duckdb_conn

def query_duckdb(sql: str, storage_path: str = ":memory:"):
    """Execute SQL query and return results as DataFrame."""
    conn = get_duckdb_conn(storage_path)
    result = conn.sql(sql)
    return result.df()

client_duckdb = OfflineClient(
    access_token=__YOUR_ACCESS_TOKEN__,
    backend=Backend.DUCKDB,
    sql_executor=query_duckdb
)

# Option 2: Simple in-memory executor
def simple_duckdb_executor(sql: str):
    return duckdb.sql(sql).df()

client_duckdb_simple = OfflineClient(
    access_token=__YOUR_ACCESS_TOKEN__,
    backend=Backend.DUCKDB,
    sql_executor=simple_duckdb_executor
)

## Trino setup

In [5]:
__YOUR_TRINO_HOST__ = "host.docker.internal"
__YOUR_TRINO_PORT__ = 8081

In [6]:
from featuremesh import OfflineClient, Backend
import pandas as pd
import trino.dbapi

def query_trino(sql: str):
    """Execute SQL query on Trino and return results as DataFrame."""
    # Configure your Trino connection details
    conn = trino.dbapi.connect(
        host=__YOUR_TRINO_HOST__,  # or host.docker.internal for docker
        port=__YOUR_TRINO_PORT__,
        user="admin",
        catalog="memory",
        schema="default"
    )
    cur = conn.cursor()
    cur.execute(sql)
    
    # Fetch results
    cols = cur.description
    rows = cur.fetchall()
    
    if len(rows) > 0:
        df = pd.DataFrame(rows, columns=[col[0] for col in cols])
        return df
    else:
        return pd.DataFrame()

client_trino = OfflineClient(
    access_token=__YOUR_ACCESS_TOKEN__,
    backend=Backend.TRINO,
    sql_executor=query_trino
)

## BigQuery setup

In [7]:
from google.cloud import bigquery
from featuremesh import OfflineClient, Backend
from libs.helpers.utils import get_bigquery_config

def query_bigquery(sql: str):
    """Execute SQL query on BigQuery and return results as DataFrame."""
    client = bigquery.Client(project=get_bigquery_config()['project'])
    return client.query(sql).to_dataframe()

client_bigquery = OfflineClient(
    access_token=__YOUR_ACCESS_TOKEN__,
    backend=Backend.BIGQUERY,
    sql_executor=query_bigquery
)

## Online setup

In [8]:
from featuremesh import OnlineClient

client_online = OnlineClient(access_token=__YOUR_ACCESS_TOKEN__)

## Run a query with method call

In [9]:
def display_results(client, sql):

    result = client.query(sql)
    
    if result.success:
        print("Query succeeded!")
        print(result.dataframe)
    else:
        print("Query failed!")
        for error in result.errors:
            print(f"Error [{error.code}]: {error.message}")
            if error.context:
                print(f"Context: {error.context}")

In [10]:
sql = """
    WITH 
        FEATURE1 := INPUT(BIGINT) 
    SELECT 
        FEATURE1 := BIND_VALUES(ARRAY[1, 2, 3]),
        FEATURE2 := FEATURE1 * 2
"""

display_results(client_duckdb, sql)
display_results(client_trino, sql)
display_results(client_bigquery, sql)
display_results(client_online, sql)

Query succeeded!
   FEATURE1  FEATURE2
0         1         2
1         2         4
2         3         6
Query succeeded!
   FEATURE1  FEATURE2
0         1         2
1         2         4
2         3         6


Query succeeded!
   FEATURE1  FEATURE2
0         1         2
1         2         4
2         3         6
Query succeeded!
   FEATURE1  FEATURE2
0         1         2
1         2         4
2         3         6


## Run a query with magic commands

In [11]:
%reload_ext featuremesh

In [12]:
%%featureql --client client_duckdb

DEFINE 
    INPUT1 := INPUT(BIGINT), 
    INPUT2 := INPUT(BIGINT),
    RESULT := INPUT1 + INPUT2,
RETURN 
    INPUT1, INPUT2, RESULT
FOR
    (INPUT1, INPUT2) := BIND_VALUES(ARRAY[ROW(2,3), ROW(2,4), ROW(2,5)])

,INPUT1,INPUT2,RESULT
0,2,3,5
1,2,4,6
2,2,5,7
